# Add transport to the MF6 flow model

In [ ]:
import sys, os
sys.path.append('../../dependencies/')
import shutil
import flopy
# import specific FloPy modules
from flopy.utils.gridgen import Gridgen 
from flopy.utils.gridintersect import GridIntersect

import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt

# for working with shapefiles
import shapefile as sf
from shapely.geometry import Polygon, LineString, MultiLineString, Point

In [ ]:
org_model_ws = os.path.join("..","..","models","sgn_mfsetup")
model_ws = org_model_ws + "_transport"
if os.path.exists(model_ws):
    shutil.rmtree(model_ws)
shutil.copytree(org_model_ws,model_ws)
sim = flopy.mf6.MFSimulation.load(sim_ws=model_ws)

In [ ]:
model_name = "sgn_50"
m = sim.get_model(model_name)

### some trickery

Just copy the gwt model from the flopy lesson

In [ ]:
gwt_name = "gwt-sgn"

In [ ]:
gwt_org_d = os.path.join("..","..","models","sgn_model_3layer")
gwt_files = [f for f in os.listdir(gwt_org_d) if gwt_name in f.lower()]
for f in gwt_files:
    shutil.copy2(os.path.join(gwt_org_d,f),os.path.join(model_ws,f))
# fix the dis
shutil.copy2(os.path.join(model_ws,model_name+".dis"),os.path.join(model_ws,gwt_name+".dis"))
# fix the nam file
lines = open(os.path.join(model_ws,gwt_name+".nam"),'r').readlines()
with open(os.path.join(model_ws,gwt_name+".nam"),'w') as f:
    for line in lines:
        if "ssm" in line.lower():
            continue
        #if "obs" in line.lower():
        #    continue
        line = line.replace("disv","dis")
        f.write(line)
# fix the mfsim nam file
lines = open(os.path.join(model_ws,"mfsim.nam"),'r').readlines()
print(lines)
with open(os.path.join(model_ws,"mfsim.nam"),'w') as f:
    for line in lines:
        f.write(line)
        if "begin models" in line.lower():
            f.write(" gwt6 {0}.nam {0}\n".format(gwt_name))
        if "begin solutiongroup" in line.lower():
            f.write(" ims6 {0}.ims {0}\n".format(gwt_name))

In [ ]:
ix = GridIntersect(m.modelgrid)

In [ ]:
source_shp = os.path.join("..","..","data","sgn","shp","real_unknown_sources.shp")
shp = sf.Reader(source_shp)
print(shp.fields)

In [ ]:
src_ij_conc = []
for s,r in zip(shp.shapes(),shp.records()):
    #print(s.points[0])
    pt = Point(*s.points[0])
    src_cell = ix.intersect(pt)
    ij_conc = list(src_cell[0][0])
    ij_conc.append(r[-2])
    if ij_conc not in src_ij_conc:
        src_ij_conc.append(ij_conc)
    #print(src_cell[0][0])

In [ ]:
src_ij_conc

In [ ]:
#rewrite the cnc file
cnc_file = os.path.join(model_ws,gwt_name+".cnc")
with open(cnc_file,'w') as f:
    f.write("BEGIN options\n  BOUNDNAMES\nEND options\n\n")
    f.write("BEGIN dimensions\nMAXBOUND {0}\nEND dimensions\n\n".format(len(src_ij_conc)))
    f.write("BEGIN period 1\n")
    for src_rec in src_ij_conc:
        line = "1 {0} {1} {2}".format(src_rec[0]+1,src_rec[1]+1,src_rec[2])
        if src_rec[2] == 200:
            line += " conc-1\n"
        else:
            line += " conc-2\n"
        f.write(line)
    f.write("END period 1\n")

            
    

In [ ]:
obs_shp = os.path.join("..","..","data","sgn","shp","Target_L1_sept2019_head_PCE.shp")
shp = sf.Reader(obs_shp)
print(shp.fields)

In [ ]:
cobs_ij = []
for s,r in zip(shp.shapes(),shp.records()):
    #print(s.points[0])
    pt = Point(*s.points[0])
    src_cell = ix.intersect(pt)
    #print(len(src_cell))
    if len(src_cell) == 0:
        print("out of grid",r[0])
        continue
    #print(src_cell)
    ij = list(src_cell[0][0])
    if ij not in cobs_ij:
        ij.append(r[0].lower())
        cobs_ij.append(ij)
    else:
        print("same cell",ij,r[0],[r for r in ij_conc if r == ij])

In [ ]:
cobs_ij

In [ ]:
with open(os.path.join(model_ws,gwt_name+".obs"),'w') as f:
    f.write("BEGIN options\n DIGITS 15\n PRINT_INPUT\nEND options\n\n")
    f.write("BEGIN continuous FILEOUT conc_obs.csv\n")
    for rec in cobs_ij:
        f.write("{0} CONCENTRATION 1 {1} {2}\n".format(rec[2],int(rec[0])+1,int(rec[1])+1))
    f.write("END continuous FILEOUT conc_obs.csv\n\n")